### Task 1: Model Implementation (80% Marks)

Implement your model that you want to submit by completing the following functions:
* `__init__`: The constructor for Model class.
* `fit`: Fit/train the model using the input data. You may perform data handling and preprocessing here before training your model.
* `predict`: Predict using the model. If you perform data handling and preprocessing in the `fit` function, then you may want to do the same here.

#### Dependencies

It is crucial to note that your model may rely on specific versions of Python packages, including:

* Python 3.10
* Numpy version 1.23
* Pandas version 1.4
* Scikit-Learn version 1.1
* PyTorch version 1.12
* Torchvision version 0.13

To prevent any compatibility issues or unexpected errors during the execution of your code, ensure that you are using the correct versions of these packages. You can refer to `environment.yml` for a comprehensive list of packages that are pre-installed in Coursemology and can be used by your model. Note that if you do end up using libraries that are not installed on Coursemology, you might see an error like:

"Your code failed to evaluate correctly. There might be a syntax error, or perhaps execution failed to complete within the allocated time and memory limits."

#### Model Template

Note that you should copy and paste the code below *directly* into Coursemology for submission. You should probably test the code in this notebook on your local machine before uploading to Coursemology and using up an attempt. 

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

class CNN(nn.Module):
    
    def __init__(self, classes, drop_prob):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.bn1 = nn.BatchNorm2d(32)  
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.bn2 = nn.BatchNorm2d(64)  
        self.fc1 = nn.Linear(64 * 2 * 2, 64)
        self.bn3 = nn.BatchNorm1d(64)  
        self.fc2 = nn.Linear(64, classes)
        self.leaky_relu = nn.LeakyReLU(0.1)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout2d(drop_prob)

    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.leaky_relu(x)
        x = self.max_pool(x)
        x = self.drop(x)
        x = self.bn2(self.conv2(x))
        x = self.leaky_relu(x)
        x = self.max_pool(x)
        x = self.drop(x)
        x = torch.flatten(x, 1)
        x = self.bn3(self.fc1(x))
        x = self.leaky_relu(x)
        x = self.fc2(x)
        return x
    

class DataLoaderHeler(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label

class Model:  
    """
    This class represents an AI model.
    """
    
    def __init__(self):
        self.model = CNN(classes=3, drop_prob=0.4)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.005)
        self.loss_fn = nn.CrossEntropyLoss()
    

    def image_replacing_nan(self, images):
        nan_mask = np.isnan(images)
        means = np.nanmean(images, axis=(2, 3), keepdims=True)  
        images[nan_mask] = np.broadcast_to(means, images.shape)[nan_mask]
        clipped_images = np.clip(images, 0, 255)
        return clipped_images
        
    
    def standardising_images(self, images):
        return images / 255.0
    
    def images_labels_filter_nan(self, images, labels):
        not_nan_indices = ~np.isnan(labels)
        filtered_images = images[not_nan_indices]
        filtered_labels = labels[not_nan_indices]
        return filtered_images, filtered_labels
    

    def data_processing(self, images, labels):
        images_with_no_nan = self.image_replacing_nan(images)
        images_standardized = self.standardising_images(images_with_no_nan)
        processed_images, processed_labels = self.images_labels_filter_nan(images_standardized, labels)
        
        return processed_images, processed_labels

    

    def fit(self, X, y):
        """
        Train the model using the input data.
        
        Parameters
        ----------
        X : ndarray of shape (n_samples, channel, height, width)
            Training data.
        y : ndarray of shape (n_samples,)
            Target values.
            
        Returns
        -------
        self : object
            Returns an instance of the trained model.
        """
        X_processed, y_processed = self.data_processing(X, y) 
        X_processed = torch.tensor(X_processed, dtype=torch.float32)
        y_processed = torch.tensor(y_processed, dtype=torch.long)
        
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.RandomAffine(degrees=5, translate=(0.1, 0.1))
        ])

        dataset = DataLoaderHeler(X_processed, y_processed, transform=transform)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
        self.model.train()
        losses = []
        for epoch in range(60):
            for batch_X, batch_y in dataloader:
                self.optimizer.zero_grad()
                output = self.model(batch_X)
                loss = self.loss_fn(output, batch_y)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                self.optimizer.step()
                loss += loss.item()

            loss = loss / len(dataloader)
            losses.append(loss)
            print ("Epoch: {}, Loss: {}".format(epoch, loss))

    def predict(self, X):
        """
        Use the trained model to make predictions.
        
        Parameters
        ----------
        X : ndarray of shape (n_samples, channel, height, width)
            Input data.
            
        Returns
        -------
        ndarray of shape (n_samples,)
        Predicted target values per element in X.
           
        """
        # TODO: Replace the following code with your own prediction code.
        X = self.image_replacing_nan(X)
        X = self.standardising_images(X)
        #print("X with no nan in predict:", X)
        X = torch.tensor(X, dtype=torch.float32) 
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X)
            #print("Pred in model: ", predictions)
            return torch.argmax(predictions, dim=1)     

#### Local Evaluation

You may test your solution locally by running the following code. Do note that the results may not reflect your performance in Coursemology. You should not be submitting the code below in Coursemology. The code here is meant only for you to do local testing.

In [2]:
# Import packages
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [3]:
# Load data
with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']

In [4]:
# Split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Filter test data that contains no labels
# In Coursemology, the test data is guaranteed to have labels
nan_indices = np.argwhere(np.isnan(y_test)).squeeze()
mask = np.ones(y_test.shape, bool)
mask[nan_indices] = False
X_test = X_test[mask]
y_test = y_test[mask]
# Train and predict
model = Model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_pred)
# Evaluate model predition
# Learn more: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
print("F1 Score (macro): {0:.2f}".format(f1_score(y_test, y_pred, average='macro'))) # You may encounter errors, you are expected to figure out what's the issue.

/Users/xiaoyangchen/anaconda3/envs/cs2109s-2310-final/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Epoch: 0, Loss: 0.006227259058505297
Epoch: 1, Loss: 0.015650637447834015
Epoch: 2, Loss: 0.005333427805453539
Epoch: 3, Loss: 0.008770504035055637
Epoch: 4, Loss: 0.010700362734496593
Epoch: 5, Loss: 0.006714912597090006
Epoch: 6, Loss: 0.02091103605926037
Epoch: 7, Loss: 0.008979168720543385
Epoch: 8, Loss: 0.010312584228813648
Epoch: 9, Loss: 0.015219288878142834
Epoch: 10, Loss: 0.006562580354511738
Epoch: 11, Loss: 0.012572119012475014
Epoch: 12, Loss: 0.009559981524944305
Epoch: 13, Loss: 0.01793425716459751
Epoch: 14, Loss: 0.01622496172785759
Epoch: 15, Loss: 0.010199349373579025
Epoch: 16, Loss: 0.005586152896285057
Epoch: 17, Loss: 0.007257682271301746
Epoch: 18, Loss: 0.012815149500966072
Epoch: 19, Loss: 0.006799270864576101
Epoch: 20, Loss: 0.008297600783407688
Epoch: 21, Loss: 0.010762771591544151
Epoch: 22, Loss: 0.010082882829010487
Epoch: 23, Loss: 0.008948263712227345
Epoch: 24, Loss: 0.007411038503050804
Epoch: 25, Loss: 0.012644109316170216
Epoch: 26, Loss: 0.005731

#### Grading Scheme

Your code implementation will be graded based on its performance ([**Macro F1 Score**](http://iamirmasoud.com/2022/06/19/understanding-micro-macro-and-weighted-averages-for-scikit-learn-metrics-in-multi-class-classification-with-example/)*) in the contest. Your model will be trained with the data that we provided you with this assesment. We will use score cutoffs that we will decide after the contest to determine your marks.

The performance of your model will be determined by a separate test data set, drawn from the same population as the training set, but not provided to you earlier. The marks you will receive will depend on the **Macro F1 Score** of the predictions:

* If your score is above the mean or median, you can expect to receive decent marks. 
* If your score is higher than the 75th percentile, you are likely to receive good marks. 
* If you achieve a score above the 90th percentile (top 10%), you will likely receive full marks.

Throughout the contest, we will provide periodic updates on the distribution of the score of student submissions in the official forum thread (see Overview) based on the **public test case**, which test the performance of the model on **a small subset of data from the hidden test data**. You can use these updates to estimate your relative standing, compared to your peers. 

<b>*) Macro F1 Score:</b> F1 score for multi-class classification computed by taking the average of all the per-class F1 score

### Task 2: Scratch Pad (20% Marks)

Fill up the `scratchpad.ipynb` with your working. 

In the **"Report" section**, write a report that explain the thought process behind your solution, and convince us that you have understood the concepts taught in class and can apply them. The report should cover data exploration and preparation, data preprocessing, modeling, and evaluation. The final solution and any alternative approaches that were tried but did not work may also be documented. The length of the report should be approximately equivalent to **1-2 pages of A4 paper (up to 1,000 words)**.

#### Grading Scheme

The report will be graded based on the reasonability and soundness of the approach you take, your understanding of the data, and your final solution. If you do not make any errors in your approach, reasoning/understanding, and conclusion, you can expect to receive full marks. This part is meant to be "standard", and is only for us to do a quick sanity check that you actually did the work required to come up with the model you submitted.

### Submission

Once you are done, please submit your work to Coursemology, by copying the right snippets of code into the corresponding box that says 'Model Implementation', and click 'Save Draft'. You can still make changes after you save your submission.

When submitting your model, the `fit` function will be called to train your model with the **data that we have provided you**. Due to the inherent stochasticity of the training process, **your model's performance may vary across different runs**. To ensure deterministic results, you can set a fixed random seed in your code. After the training is completed, the `predict` function will be used to evaluate your model. The evaluation of your model will be based on two test cases: 
1. **Public test cases, containing a small portion of the test data**, that allows you to **estimate** your score. 
2. **Evaluation test cases containing the remaining test data** (which you will not be able to see) by which we will evaluate your model. 

Your score in the public test case may not reflect your actual score. **Note that running all test cases can take up to 5 minutes to complete, and you have a maximum of 20 attempts.** We only provide you with a limited number of tries because we do not want you to spam our autograder. 

Finally, when you are satisfied with your submission, you can finalize it by clicking "Finalize submission.". <span style="color:red">**Note that once you have finalized your submission, it is considered to be submitted for grading, and no further changes can be made**.</span>